## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


2023-09-12 20:22:44.330378: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN','NAME'])
application_df


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts = pd.DataFrame(application_counts)
few_counts = application_counts[application_counts['APPLICATION_TYPE']<500]
application_types_to_replace = list(few_counts.index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_counts = pd.DataFrame(classification_counts)
few_counts = classification_counts[classification_counts['CLASSIFICATION']<600]
classifications_to_replace = list(few_counts.index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
dummied_df = pd.get_dummies(application_df)
dummied_df


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
# Split our preprocessed data into our features and target arrays
y = dummied_df.IS_SUCCESSFUL.values
X = dummied_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

### Optimization Attempt: 1 ==> Standard Setup

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

# First hidden layer (there are 44 training columns)
nn_model.add(tf.keras.layers.Dense(units=44, activation="relu", input_dim=44))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=44, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 44)                1980      
                                                                 
 dense_1 (Dense)             (None, 44)                1980      
                                                                 
 dense_2 (Dense)             (None, 1)                 45        
                                                                 
Total params: 4005 (15.64 KB)
Trainable params: 4005 (15.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [13]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=10)


Epoch 1/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5738 - accuracy: 0.7169
Epoch 2/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5548 - accuracy: 0.7314
Epoch 3/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5514 - accuracy: 0.7317
Epoch 4/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5498 - accuracy: 0.7330
Epoch 5/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5484 - accuracy: 0.7332
Epoch 6/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7339
Epoch 7/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accuracy: 0.7339
Epoch 8/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7346
Epoch 9/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5451 - accuracy: 0.7343
Epoch 10/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5452 - accuracy: 0.7353

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5537 - accuracy: 0.7241 - 552ms/epoch - 2ms/step
Loss: 0.5537358522415161, Accuracy: 0.7240816354751587


In [15]:
# Export our model to HDF5 file
nn_model.save('charity_app_success_model_v1.h5')


/Users/bearscomputer/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Optimization Attempt: 2 ==> Modification: add a third hidden layer with tanh

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model_v2 = tf.keras.models.Sequential()

# First hidden layer (there are 44 training columns)
nn_model_v2.add(tf.keras.layers.Dense(units=44, activation="relu", input_dim=44))

# Second hidden layer
nn_model_v2.add(tf.keras.layers.Dense(units=44, activation="relu"))

# THIRD hidden layer !!!!!!!!!!
nn_model_v2.add(tf.keras.layers.Dense(units=44, activation="tanh"))

# Output layer
nn_model_v2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_model_v2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model_v2 = nn_model_v2.fit(X_train_scaled, y_train, epochs=10)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_v2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn_model_v2.save('charity_app_success_model_v2.h5')


Epoch 1/10
804/804 [==============================] - 3s 2ms/step - loss: 0.5665 - accuracy: 0.7216
Epoch 2/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5533 - accuracy: 0.7311
Epoch 3/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5504 - accuracy: 0.7311
Epoch 4/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5494 - accuracy: 0.7322
Epoch 5/10
804/804 [==============================] - 2s 3ms/step - loss: 0.5483 - accuracy: 0.7338
Epoch 6/10
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7325
Epoch 7/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5459 - accuracy: 0.7343
Epoch 8/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5457 - accuracy: 0.7346
Epoch 9/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5450 - accuracy: 0.7344
Epoch 10/10
804/804 [==============================] - 1s 2ms/step - loss: 0.5449 - accuracy: 0.7338

### Optimization Attempt: 3 ==> Modification: use keras auto-tuner

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=90,
        step=5), activation=activation, input_dim=X_train_scaled.shape[1]))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

In [18]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 5,
 'num_layers': 2,
 'units_0': 7,
 'units_1': 9,
 'units_2': 1,
 'units_3': 9,
 'units_4': 3,
 'units_5': 9,
 'tuner/epochs': 17,
 'tuner/initial_epoch': 6,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0054'}

In [19]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5562 - accuracy: 0.7272 - 610ms/epoch - 2ms/step
Loss: 0.5561837553977966, Accuracy: 0.7272303104400635


## Report

The goal of this analysis is to determine, based on the features of different charity applications, if we can reliably predict which applications will be successful or not. This is useful, because it allows us to identify applications that could require further information, support, or redirection in order to increase potential success.

#### Data Preprocessing:
The target variable for our model is 'IS_SUCCESSFUL', where 1 indicates that the application was successful, and 0 indicates that it was not.\
The features of each application that we considered in our model are as follows: \
\
'APPLICATION_TYPE',\
'AFFILIATION',\
'CLASSIFICATION',\
'USE_CASE',\
'ORGANIZATION',\
'STATUS',\
'INCOME_AMT',\
'SPECIAL_CONSIDERATIONS',\
'ASK_AMT',\
\
Variables that we removed from consideration were 'EIN' and 'NAME' of the organization completing the application, as these are incidental properties of the application, and should have no bearing on the outcome (ideally).


#### The Model
Multiple unsuccessful models were attempted, resulting in an overall accuracy rate of around 72%. In an effort to try to optimize, I applied a keras hyperparameter tuner, but after an extended period of trialing, accuracy for various models topped out at just under 73%, a negligible improvement that still fell short of targeted performance of 75%.\

#### Summary
Based on the near-success of the model, my claim is that there is insufficient data in order to sufficiently (over 75% reliable) determine the success of an application. It is likely that there are independent variables not captured here that have meaningful impacts on the success of the applicant's venture. For example, the size of the organization, economic context, time of year of application, or number/success of previous applications. I recommend that we partner with the team developing the application to add further features for tracking, study, and modelling. Additionally, I recommend we work with that team to mine available data resevoirs in order to expand already-compelted applications.